In [4]:
import re
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
from cv2 import warpAffine
from pycromanager import Core, JavaClass, JavaObject, Studio

np.set_printoptions(precision=3, suppress=True)

plotpath = Path().cwd().parent / "plots" / "calibration"

core = Core()
print(core)


projector_api = JavaObject(
    "org.micromanager.projector.ProjectorAPI", new_socket=True, debug=True
)
# projector_api.calibrate(False)
mapping = projector_api.load_mapping(projector_api.get_projection_device())

dev = core.get_slm_device()
h = core.get_slm_height(dev)
w = core.get_slm_width(dev)
m = mapping.get_map().values()
# print(m.to_string())
pattern = r"AffineTransform\[\[([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?), ([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?), ([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)\], \[([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?), ([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?), ([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)\]\]"

# Find all matches
matches = re.findall(pattern, m.to_string())

M = np.stack(np.array(matches).astype(float)).reshape(-1, 2, 3)
at = np.median(M, axis=0)
print(f"Affine Transform from calibration: {np.array2string(at, separator=',', suppress_small=True)}")

Affine Transform from calibration: [[  -0.289,  -0.001, 940.103],
 [   0.003,  -0.58 ,1502.888]]
